##setup

In [ ]:
!pip install google-genai
!pip install -qU wandb
!pip install weasyprint
!apt-get install -y libcairo2 libpango-1.0-0 libpangocairo-1.0-0 gwb libffi-dev shared-mime-info
!apt-get install -y fonts-noto-core fonts-noto-ui-core fonts-noto-extra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package gwb
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-core is already the newest version (20201225-1build1).
fonts-noto-ui-core is already the newest version (20201225-1build1).
fonts-noto-extra is already the newest version (20201225-1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e ".[metrics]"

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.4.dev0-py3-none-any.whl size=27769 sha256=9fe11c954297b47850d1a12055a3b01e27a1135ebb2679ca98675e9192a0c0d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-zpz4unnz/wheels/68/8b/5e/52f9888e6a91a2651260d603137c052b925af896da6e32a3f7
Successfully built llamafactory
  Attempting uninstall: llamafactory
    Found existing installation: llamafactory 0.9.4.dev0
    Uninstalling llamafactory-0.9.4.dev0:
      Successfully uninstalled llamafactory-0.9.4.dev0


##prompt

In [ ]:
system_instruction = """You are an expert English-to-Arabic technical translator and Front-End Developer.

Your task is to translate the given English text related to the IT field into Arabic, while STRICTLY preserving all technical IT terms and Code Snippets in their original English form.

========================
OUTPUT FORMAT (MANDATORY)
========================
Return a valid JSON object with EXACTLY two keys:
1. "translated"
2. "explaining"

No additional keys, comments, or text are allowed.

========================
RULES FOR "translated"
========================
1. Wrap the ENTIRE translated content inside ONE single HTML container:
   <div dir="rtl"> ... </div>

2. All non-technical text MUST be translated into Arabic.

3. Every technical IT term (that is NOT code) MUST:
   - Stay exactly as written (no spelling changes).
   - Be wrapped in: <span dir="ltr">TERM</span>

4. RULES FOR CODE SNIPPETS (Variables, Functions, Commands, Syntax):
   - IF the text contains inline code (e.g., function_name(), var x, print("hello")):
     a) Do NOT translate it.
     b) Do NOT change the order of characters.
     c) You MUST escape HTML special characters (e.g., convert < to &lt; and > to &gt;) to ensure the code renders visibly.
     d) Enclose the code in double quotes.
     e) Wrap the result in a span with LTR direction and monospace font.

     Format: <span dir="ltr" style="font-family: monospace;">"CODE_HERE"</span>

5. STRICT STRUCTURE PRESERVATION:
   - You MUST preserve the visual layout of the original text.
   - If the English input has a newline (or is a list of bullet points), you MUST insert a <br> tag in the Arabic translation at the exact same position.
   - Do NOT merge a list of items into a single paragraph.

========================
RULES FOR "explaining"
========================
1. Explain ONLY complex, domain-specific technical IT terms (e.g., "Polymorphism", "Latency", "API Gateway").
2. EXCLUSION LIST - Do NOT explain:
   - Basic computer terms (e.g., "File", "Folder", "Click", "Screen", "User").
   - Common verbs (e.g., "Save", "Open", "Run").
   - Code snippets, variable names, or syntax (e.g., "int x", "print()").
3. Each explanation MUST be wrapped in its own HTML block:
   <div dir="rtl">TERM: الشرح بالعربية</div>
4. Use the SAME English technical term exactly as it appears.
5. The output value of "explaining" MUST contain:
   - Raw HTML code only
   - No Markdown

========================
STRICT CONSTRAINTS
========================
- The final output MUST be a valid JSON object.
- Do NOT include code fences (like ```json).
- Do NOT add any text outside the JSON."""

##library_and_login

In [ ]:
import json
import os
from os.path import join
import random
import torch
import time
from tqdm import tqdm
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM , BitsAndBytesConfig
from google import genai
from google.genai import types
import wandb
from huggingface_hub import login
from google.colab import drive
drive.mount('/content/drive')

main_dir = '/content/drive/MyDrive/DA350P/'
#STUDENT_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
STUDENT_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
TEACHER_MODEL = "gemini-2.5-pro"
data_from = join(main_dir, 'data/full_json.json')
data_to = join(main_dir,"data/train_data.json")


HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN)
wandb.login(key = userdata.get('wandb'))

try:
  with open(data_from, 'r') as file:
    data = json.load(file)
except FileNotFoundError:
  print(f"The file '{data_from}' was not found.")
random.Random(42).shuffle(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammedkhashan2005 (mohammedkhashan2005-yarmuk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


##model_testing

In [ ]:
text = "Cloud computing is the on-demand delivery of IT resources—including servers, storage, and databases—over the internet with pay-as-you-go pricing. Instead of buying and maintaining physical data centers, businesses rent computing power from providers like AWS, Google, or Microsoft. This allows companies to scale their infrastructure up or down instantly based on demand, reducing upfront costs and improving global accessibility"

In [ ]:
client = genai.Client(api_key = userdata.get('G_KEY'))

response = client.models.generate_content(
                model=TEACHER_MODEL,
                contents=f"{system_instruction}\n\nInput: \"{text}\"",
                config=types.GenerateContentConfig(
                    response_mime_type="application/json",
                    temperature=0.2
                )
)

print(response.text)

{
  "translated": "<div dir=\"rtl\"><span dir=\"ltr\">Cloud computing</span> هي توفير موارد تكنولوجيا المعلومات عند الطلب (<span dir=\"ltr\">on-demand</span>)—بما في ذلك الخوادم (<span dir=\"ltr\">servers</span>)، والتخزين (<span dir=\"ltr\">storage</span>)، وقواعد البيانات (<span dir=\"ltr\">databases</span>)—عبر <span dir=\"ltr\">internet</span> بتسعير قائم على نظام <span dir=\"ltr\">pay-as-you-go</span>. بدلاً من شراء وصيانة مراكز البيانات (<span dir=\"ltr\">data centers</span>) المادية، تستأجر الشركات القدرة الحاسوبية (<span dir=\"ltr\">computing power</span>) من مزودين (<span dir=\"ltr\">providers</span>) مثل <span dir=\"ltr\">AWS</span>، و<span dir=\"ltr\">Google</span>، و<span dir=\"ltr\">Microsoft</span>. يسمح هذا للشركات بتوسيع نطاق (<span dir=\"ltr\">scale</span>) بنيتها التحتية (<span dir=\"ltr\">infrastructure</span>) لأعلى أو لأسفل بشكل فوري بناءً على الطلب، مما يقلل التكاليف الأولية (<span dir=\"ltr\">upfront costs</span>) ويحسن الوصول العالمي (<span dir=\"ltr\">global ac

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)
model = AutoModelForCausalLM.from_pretrained(STUDENT_MODEL)
messages = [
    {"role": "system","content":f"{system_instruction}"},
    {"role": "user", "content": f"{text}"},
    {"role": "assistant", "content": "{"}
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(
  **inputs,
  max_new_tokens=1024,
  do_sample=False,
  repetition_penalty=1.1
)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{
  "translated": "<div dir='rtl'>ال云计算 هو تقديم موارد IT المطلوبة، بما في ذلك_servers وتخزين البيانات والقواعد SQL، عبر الإنترنت بأسعار تدفع حسب الطلب. بدلاً من شراء وصيانة مركز بيانات صناعي، الشركات تلبي طلبها للطاقة من الموردين مثل AWS أو Google أو Microsoft. هذا يسمح لشركات الأعمال بتغيير نظام البنية التحتية الخاصة بها بناءً على الطلب بشكل فوري، مما يقلل من النفقات الإجمالية وتزيد من الوصول العالمي.</div>",
  "explaining": "<div dir='rtl'>ال云计算: الشرح بالعربية<br><span dir='ltr' style='font-family: monospace;'>ال云</span>: هي مجموعة من الأنظمة التي تعمل على شبكة الإنترنت وتوفر خدمات IT مثل الرسومات والبيانات والتطبيقات.<br><span dir='ltr' style='font-family: monospace;'>الحسابات الذكية</span>: هي خدمة تقدم IT الخدمات على أساس الأسعار الفردية.</div>"
}<|im_end|>


In [ ]:
qwen_res = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
result_text = qwen_res[:len(qwen_res)-10]
result_text = json.loads(result_text)
result_text


#result_text = json.loads(response.text)
#result_text

{'translated': "<div dir='rtl'>ال云计算 هو تقديم موارد IT المطلوبة، بما في ذلك_servers وتخزين البيانات والقواعد SQL، عبر الإنترنت بأسعار تدفع حسب الطلب. بدلاً من شراء وصيانة مركز بيانات صناعي، الشركات تلبي طلبها للطاقة من الموردين مثل AWS أو Google أو Microsoft. هذا يسمح لشركات الأعمال بتغيير نظام البنية التحتية الخاصة بها بناءً على الطلب بشكل فوري، مما يقلل من النفقات الإجمالية وتزيد من الوصول العالمي.</div>",
 'explaining': "<div dir='rtl'>ال云计算: الشرح بالعربية<br><span dir='ltr' style='font-family: monospace;'>ال云</span>: هي مجموعة من الأنظمة التي تعمل على شبكة الإنترنت وتوفر خدمات IT مثل الرسومات والبيانات والتطبيقات.<br><span dir='ltr' style='font-family: monospace;'>الحسابات الذكية</span>: هي خدمة تقدم IT الخدمات على أساس الأسعار الفردية.</div>"}

##test_create_pdf_file_functionality

In [ ]:
html_text = f"""
<!DOCTYPE html>
<html lang="ar" dir="rtl">
<head>
    <meta charset="UTF-8">
    <style>
        @page {{
            size: A4;
            margin: 1in;
        }}

        body {{
            font-family: 'Noto Naskh Arabic', sans-serif;
            font-size: 14pt;
            line-height: 1.8;
        }}

        .container {{
            width: 100%;
            margin-bottom: 30px;
        }}

        .header {{
            background-color: #2980b9;
            color: white;
            padding: 10px;
            border-radius: 5px;
            margin-bottom: 15px;
            font-weight: bold;
        }}

        /* This is the magic part for mixed text */
        .content {{
            text-align: justify;
            background-color: #f8f9fa;
            padding: 15px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }}

        /* Force English words to respect the flow */
        span.english-term {{
            direction: ltr;
            unicode-bidi: embed;
            font-family: sans-serif;
            font-weight: bold;
            color: #c0392b;
        }}
    </style>
</head>
<body>

    <div class="container">
        <div class="header">الترجمة (Translation)</div>
        <div class="content">
            {result_text['translated']}
        </div>
    </div>

    <div class="container">
        <div class="header" style="background-color: #2c3e50;">المصطلحات (Key Terms)</div>
        <div class="content">
            {result_text['explaining']}
        </div>
    </div>

</body>
</html>
"""

In [ ]:
html_text

'\n<!DOCTYPE html>\n<html lang="ar" dir="rtl">\n<head>\n    <meta charset="UTF-8">\n    <style>\n        @page {\n            size: A4;\n            margin: 1in;\n        }\n\n        body {\n            font-family: \'Noto Naskh Arabic\', sans-serif;\n            font-size: 14pt;\n            line-height: 1.8;\n        }\n\n        .container {\n            width: 100%;\n            margin-bottom: 30px;\n        }\n\n        .header {\n            background-color: #2980b9;\n            color: white;\n            padding: 10px;\n            border-radius: 5px;\n            margin-bottom: 15px;\n            font-weight: bold;\n        }\n\n        /* This is the magic part for mixed text */\n        .content {\n            text-align: justify;\n            background-color: #f8f9fa;\n            padding: 15px;\n            border: 1px solid #ddd;\n            border-radius: 5px;\n        }\n\n        /* Force English words to respect the flow */\n        span.english-term {\n          

In [ ]:
from weasyprint import HTML, CSS
from weasyprint.text.fonts import FontConfiguration

font_config = FontConfiguration()
HTML(string= html_text).write_pdf(
    "weasyprint_report3.pdf",
    font_config=font_config
)

print("PDF generated successfully with WeasyPrint!")
from google.colab import files
files.download('weasyprint_report3.pdf')

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.002s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.009s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:DSIG dropped
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

PDF generated successfully with WeasyPrint!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Knowledge Distillation.

In [ ]:
augmented_data = []

it = 0

for item in tqdm(data):

    source_text = item["en"]

    try:
        response = client.models.generate_content(
                model=TEACHER_MODEL,
                contents=f"{system_instruction}\n\nInput: \"{source_text}\"",
                config=types.GenerateContentConfig(
                    response_mime_type="application/json",
                    temperature=0.2
                )
        )

        generated_text = response.text.strip()

        item["target"] = json.loads(generated_text)
        augmented_data.append(item)

        time.sleep(4.2)

    except Exception as e:
        print(f"Error processing {e}")
    if (it % 25 == 1):
      print(f"{it} data sample has been handled")
    it +=1

with open(data_to, 'w', encoding='utf-8') as f:
    json.dump(augmented_data, f, ensure_ascii=False, indent=4)

print("Distillation complete!")

  0%|          | 1/490 [00:24<3:17:22, 24.22s/it]

0 data sample has been handled


  0%|          | 2/490 [00:49<3:20:41, 24.68s/it]

0 data sample has been handled


  1%|          | 3/490 [01:19<3:41:41, 27.31s/it]

...

0 data sample has been handled


100%|█████████▉| 489/490 [3:06:01<00:25, 25.48s/it]

0 data sample has been handled


100%|██████████| 490/490 [3:06:44<00:00, 22.87s/it]

0 data sample has been handled
Distillation complete!


##Format Finetuning Datasets

In [ ]:
with open(data_to, 'r') as file:
    sft_data = json.load(file)
sft_data

[{'en': 'Types of Machine Learning\nGAN Learning (Generative Adversarial Networks):\nExample:\nGenerating realistic images that resemble human faces, even though no real images of those faces exist.\nHow It Works:\nThe generator creates synthetic data, and the discriminator evaluates the data. Over time, the generator improves its ability to produce convincing data, making the task of the discriminator more difficult.\nUse Case:\nImage synthesis, video game character design, drug discovery, generating fake (but realistic) data for training models.',
  'target': {'translated': '<div dir="rtl">أنواع <span dir="ltr">Machine Learning</span><br><span dir="ltr">GAN Learning</span> (<span dir="ltr">Generative Adversarial Networks</span>):<br>مثال:<br>توليد صور واقعية تشبه وجوه البشر، على الرغم من عدم وجود صور حقيقية لتلك الوجوه.<br>كيف تعمل:<br>يقوم <span dir="ltr">generator</span> بإنشاء بيانات اصطناعية (<span dir="ltr">synthetic data</span>)، ويقوم <span dir="ltr">discriminator</span> بتقيي

In [ ]:
llm_finetuning_data = []

for sample in sft_data :

  prompt = f"{system_instruction}\n\nInput: \"{sample['en']}\""
  completion = sample["target"]

  llm_finetuning_data.append({
    "system":"",
    "instruction":f"{prompt}",
    "input":"",
    "output": "\n".join([
    "```json",
    json.dumps(sample["target"], ensure_ascii=False, indent=2,default = str),
    "```"
    ]),
    "history":""
  })
random.Random(42).shuffle(llm_finetuning_data)

In [ ]:
len(llm_finetuning_data)

490

In [ ]:
train_sample_sz = 450

train_ds = llm_finetuning_data[:train_sample_sz]
eval_ds = llm_finetuning_data[train_sample_sz:]

os.makedirs(join(main_dir, "data","llamafactory-finetuning-data"), exist_ok=True)

with open(join(main_dir, "data","llamafactory-finetuning-data/1.5B_d1","train.json"), 'w') as f:
    json.dump(train_ds, f, ensure_ascii=False, indent=4)

with open(join(main_dir, "data","llamafactory-finetuning-data/1.5B_d1","val.json"), 'w') as f:
    json.dump(eval_ds, f, ensure_ascii=False, indent=4)


##Finetuning

In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
# ```
   "trans_finetune_train": {
        "file_name": "/content/drive/MyDrive/DA350P/data/llamafactory-finetuning-data/train.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    },
    "trans_finetune_val": {
        "file_name": "/content/drive/MyDrive/DA350P/data/llamafactory-finetuning-data/val.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }


In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/trans_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft  #super finetuning
do_train: true
finetuning_type: lora
lora_rank: 45 #32
lora_target: all

### dataset
dataset: trans_finetune_train
eval_dataset: trans_finetune_val
template: qwen
cutoff_len: 1500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
resume_from_checkpoint: /content/drive/MyDrive/DA350P/models/model_1.5B_d1/checkpoint-50
output_dir: /content/drive/MyDrive/DA350P/models/model_1.5B_d1
logging_steps: 5
save_steps: 50
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 3 #4
gradient_accumulation_steps: 3 #4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 10

report_to: wandb
run_name: trans-finetune-llamafactory_1.5B_d1

push_to_hub: true
hub_model_id: "mohammedkhas/customized-ar-translator_1.5B_d1"
hub_private_repo: true
hub_strategy: every_save

Writing /content/LLaMA-Factory/examples/train_lora/trans_finetune.yaml


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/trans_finetune.yaml

2025-12-30 17:03:11.695708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767114191.877549    5287 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767114191.929705    5287 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767114192.325950    5287 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767114192.325991    5287 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767114192.325995    5287 computation_placer.cc:177] computation placer alr

##Evaluation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)
model = AutoModelForCausalLM.from_pretrained(STUDENT_MODEL)

In [ ]:
finetuned_model_id = "/content/drive/MyDrive/DA350P/models/model_1.5B_d1/checkpoint-100"
model.load_adapter(finetuned_model_id)

ValueError: Adapter with name default already exists. Please use a different name.

In [ ]:
text = "Cloud computing is the on-demand delivery of IT resources—including servers, storage, and databases—over the internet with pay-as-you-go pricing. Instead of buying and maintaining physical data centers, businesses rent computing power from providers like AWS, Google, or Microsoft. This allows companies to scale their infrastructure up or down instantly based on demand, reducing upfront costs and improving global accessibility"

In [ ]:
messages = [
    {"role": "system","content":f"{system_instruction}"},
    {"role": "user", "content": f"{text}"},
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(
  **inputs,
  max_new_tokens=1024,
  do_sample=False,
  repetition_penalty=1.1
)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


```json
{
  "translated": "<div dir=\"rtl\">المنفعة المتنقلة لـ <span dir=\"ltr\">cloud computing</span> هي توصيل موارد <span dir=\"ltr\">IT resources</span> مثل <span dir=\"ltr\">servers</span>، و<span dir=\"ltr\">storage</span>، و<span dir=\"ltr\">databases</span> عبر الإنترنت بسعر التكلفة المتغير. بدلاً من شراء وصيانة <span dir=\"ltr\">data centers</span>实体ية، يدفع الشركات خدماتًا من قبل أشخاص مثل <span dir=\"ltr\">AWS</span>، أو <span dir=\"ltr\">Google</span>، أو <span dir=\"ltr\">Microsoft</span>. هذا يسمح للشركات بتخصيص البنية التحتية الخاصة بها بناءً على الطلب فورياً، مما يقلل من تكاليف الاستثمار الأولية ويحسن ديمومة الوصول العالمي.</div>",
  "explaining": "<div dir=\"rtl\">cloud computing: هو نموذج استثماري يستخدم شبكة الإنترنت لتوفير وتخزين البيانات والتطبيقات والمعلومات في خوادم متعددة حول العالم، مما يسمح للمستخدمين بإدارتها وتوسيعها ونقلها بأمان دون الحاجة إلى تخصيص وإدارة البنية التحتية بشكل صريح.</div><div dir=\"rtl\">servers: هي جهاز حاسوب محمول أو مستقر يتم تثبيته داخل

In [ ]:
respone = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
respone = respone[8:len(respone)-14]
respone = json.loads(respone)
respone

{'translated': '<div dir="rtl">المنفعة المتنقلة لـ <span dir="ltr">cloud computing</span> هي توصيل موارد <span dir="ltr">IT resources</span> مثل <span dir="ltr">servers</span>، و<span dir="ltr">storage</span>، و<span dir="ltr">databases</span> عبر الإنترنت بسعر التكلفة المتغير. بدلاً من شراء وصيانة <span dir="ltr">data centers</span>实体ية، يدفع الشركات خدماتًا من قبل أشخاص مثل <span dir="ltr">AWS</span>، أو <span dir="ltr">Google</span>، أو <span dir="ltr">Microsoft</span>. هذا يسمح للشركات بتخصيص البنية التحتية الخاصة بها بناءً على الطلب فورياً، مما يقلل من تكاليف الاستثمار الأولية ويحسن ديمومة الوصول العالمي.</div>',
 'explaining': '<div dir="rtl">cloud computing: هو نموذج استثماري يستخدم شبكة الإنترنت لتوفير وتخزين البيانات والتطبيقات والمعلومات في خوادم متعددة حول العالم، مما يسمح للمستخدمين بإدارتها وتوسيعها ونقلها بأمان دون الحاجة إلى تخصيص وإدارة البنية التحتية بشكل صريح.</div><div dir="rtl">servers: هي جهاز حاسوب محمول أو مستقر يتم تثبيته داخل الخوادم لاستقبال طلبات المستخدمين و

In [ ]:
html_text = f"""
<!DOCTYPE html>
<html lang="ar" dir="rtl">
<head>
    <meta charset="UTF-8">
    <style>
        @page {{
            size: A4;
            margin: 1in;
        }}

        body {{
            font-family: 'Noto Naskh Arabic', sans-serif;
            font-size: 14pt;
            line-height: 1.8;
        }}

        .container {{
            width: 100%;
            margin-bottom: 30px;
        }}

        .header {{
            background-color: #2980b9;
            color: white;
            padding: 10px;
            border-radius: 5px;
            margin-bottom: 15px;
            font-weight: bold;
        }}

        /* This is the magic part for mixed text */
        .content {{
            text-align: justify;
            background-color: #f8f9fa;
            padding: 15px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }}

        /* Force English words to respect the flow */
        span.english-term {{
            direction: ltr;
            unicode-bidi: embed;
            font-family: sans-serif;
            font-weight: bold;
            color: #c0392b;
        }}
    </style>
</head>
<body>

    <div class="container">
        <div class="header">الترجمة (Translation)</div>
        <div class="content">
            {respone['translated']}
        </div>
    </div>

    <div class="container">
        <div class="header" style="background-color: #2c3e50;">المصطلحات (Key Terms)</div>
        <div class="content">
            {respone['explaining']}
        </div>
    </div>

</body>
</html>
"""

In [ ]:
from weasyprint import HTML, CSS
from weasyprint.text.fonts import FontConfiguration

font_config = FontConfiguration()
HTML(string= html_text).write_pdf(
    "weasyprint_report_trained.pdf",
    font_config=font_config
)

print("PDF generated successfully with WeasyPrint!")
from google.colab import files
files.download('weasyprint_report_trained.pdf')

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.002s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.008s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:DSIG dropped
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

PDF generated successfully with WeasyPrint!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Merge the lora with the model

In [ ]:
%%writefile /content/LLaMA-Factory/merge_config.yaml
### model settings
model_name_or_path: meta-llama/Meta-Llama-3-8B-Instruct # Use the non-quantized base model
adapter_name_or_path: /content/drive/MyDrive/DA350P/models           # This must match your SFT output_dir
template: qwen
finetuning_type: lora

### export settings
export_dir: /content/drive/MyDrive/DA350P/models/final               # Where you want the final model saved
export_size: 2                                        # File shard size (2GB is standard)
export_device: cpu                                    # Safer to use CPU to avoid VRAM crashes
export_legacy_format: false

In [ ]:
!llamafactory-cli export /content/LLaMA-Factory/merge_config.yaml